In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
# from bs4 import BeautifulSoup
import os
# import zipfilea
from scipy.spatial import cKDTree
# import osmnx as ox
# import multiprocessing

not all the pfas necessarily goes to the waste transfer stations.  
only if the main landfill is more distant than the waste transfer

In [5]:
year = 2020

tiger_tracts = gpd.read_file('tiger/tiger_tracts_{}.geojson'.format(year))
tiger_tracts = tiger_tracts[['GEOID', 'geometry']]

landfill = pd.read_csv('landfill_{}.csv'.format(year))
before = landfill[landfill['Year Landfill Opened'] > year]
after = landfill[landfill['Closure_Year'] < year]
# remove the before_2015 and after_2015 from the landfill_2015
landfill = landfill.drop(before.index)
landfill = landfill.drop(after.index)
landfill = landfill[~landfill['State'].isin(['GU', 'VI'])]
landfill = landfill.dropna(subset=['Latitude', 'Longitude'])
landfill.drop_duplicates(subset=['Latitude', 'Longitude'], inplace=True)
landfill.reset_index(inplace=True, drop=True)
# create a geodataframe from the landfill
landfill_gdf = gpd.GeoDataFrame(landfill, geometry=gpd.points_from_xy(landfill.Longitude, landfill.Latitude), crs='EPSG:4269')
landfill_gdf['x'] = landfill_gdf.geometry.x
landfill_gdf['y'] = landfill_gdf.geometry.y

In [6]:
pfas = pd.read_csv('PFAS_Census tracts_{}_Peter.csv'.format(year))
pfas['GEOID'] = pfas['GEO_ID'].str[9:]
pfas = pfas[['GEOID', 'PFAS_CT_{}'.format(year)]]
pfas_gdf = pfas.merge(tiger_tracts, on='GEOID', how='left')
pfas_gdf = gpd.GeoDataFrame(pfas_gdf, geometry=pfas_gdf.geometry, crs='EPSG:4269')
pfas_gdf['centroid'] = pfas_gdf.geometry.centroid
pfas_gdf['x'] = pfas_gdf.centroid.x
pfas_gdf['y'] = pfas_gdf.centroid.y
pfas_gdf = pfas_gdf.dropna(subset=['x', 'y'])

C:\Users\hgazmeh\AppData\Local\Temp\2\ipykernel_9352\1357485647.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pfas_gdf['centroid'] = pfas_gdf.geometry.centroid
C:\Users\hgazmeh\AppData\Local\Temp\2\ipykernel_9352\1357485647.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pfas_gdf['x'] = pfas_gdf.centroid.x
C:\Users\hgazmeh\AppData\Local\Temp\2\ipykernel_9352\1357485647.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pfas_gdf['y'] = pfas_gdf.centroid.y


### get the waste transfer stations

In [3]:
landfill_trans = pd.read_csv('Transfer Station.csv')
# landfill_trans = landfill_trans[['Latitude','Longitude','Landfill','Transfer Station','DDRT_ID','x','y']]
landfill_trans = landfill_trans.dropna(subset=['Latitude', 'Longitude'])
landfill_trans.drop_duplicates(subset=['Latitude', 'Longitude'], inplace=True)
landfill_trans.reset_index(inplace=True, drop=True)
# add a id column
landfill_trans['landfill_trans_id'] = landfill_trans.index + 1 
# create a geodataframe from the landfill
landfill_trans_gdf = gpd.GeoDataFrame(landfill_trans, geometry=gpd.points_from_xy(landfill_trans.Longitude, landfill_trans.Latitude), crs='EPSG:4269')
landfill_trans_gdf['x'] = landfill_trans_gdf.geometry.x
landfill_trans_gdf['y'] = landfill_trans_gdf.geometry.y

waste_transfer_stations = landfill_trans_gdf[landfill_trans_gdf['Landfill'] == 'No']
waste_transfer_stations.reset_index(inplace=True, drop=True)
# waste_transfer_stations.to_file('waste_transfer_stations.geojson', driver='GeoJSON')
# waste_transfer_stations.to_csv('waste_transfer_stations.csv', index=False)

### get the main landfill facilities

In [18]:
landfill_gdf_sub = landfill_gdf[['City', 'State', 'Zip Code', 'Latitude', 'Longitude', 'Landfill ID']]

additional_landfills = landfill_trans_gdf[landfill_trans_gdf['Landfill'] == 'Yes']
additional_landfills  = additional_landfills[['City', 'State', 'Zip', 'Latitude', 'Longitude', 'DDRT_ID']]
additional_landfills.rename(columns={'Zip':'Zip Code'}, inplace=True)

# concat two dataframes into one
main_landfills = pd.concat([landfill_gdf_sub, additional_landfills], axis=0)
main_landfills['main_landfill_ID'] = main_landfills['Landfill ID'].fillna(main_landfills['DDRT_ID'])
main_landfills_gdf = gpd.GeoDataFrame(main_landfills, geometry=gpd.points_from_xy(main_landfills.Longitude, main_landfills.Latitude), crs='EPSG:4269')
main_landfills_gdf.drop_duplicates(subset=['Latitude', 'Longitude'], inplace=True)
main_landfills_gdf.reset_index(inplace=True, drop=True)
# main_landfills_gdf.to_file("main_landfills_gdf.geojson", driver='GeoJSON')
# main_landfills_gdf.to_csv("main_landfills_gdf.csv", index=False)

## assignment  
- we create a dataframe of all the main landfills and the waste transfer stations  
- and assign the census tracts to them  
- if a census tract is assigned to the waste transfer station, we perform a second assignment

1- dataframe of all the main landfills

In [43]:
waste_transfer_stations['facility_id'] = waste_transfer_stations['DDRT_ID']
waste_transfer_stations['facility_label'] = 'WTS'
main_landfills_gdf['facility_id'] = main_landfills_gdf['main_landfill_ID']
main_landfills_gdf['facility_label'] = 'MLF'

all_facilities = pd.concat([waste_transfer_stations[['Latitude', 'Longitude', 'facility_id', 'facility_label']], main_landfills_gdf[['Latitude', 'Longitude', 'facility_id', 'facility_label']]], axis=0)
all_facilities = gpd.GeoDataFrame(all_facilities, geometry=gpd.points_from_xy(all_facilities.Longitude, all_facilities.Latitude), crs='EPSG:4269')
all_facilities.reset_index(inplace=True, drop=True)
# all_facilities.to_file('all_facilities.geojson', driver='GeoJSON')
# all_facilities.to_csv('all_facilities.csv', index=False)

c:\Users\hgazmeh\Anaconda3\envs\myenv\lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\hgazmeh\Anaconda3\envs\myenv\lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


2- assignment of all the facilities to the cts  
done using the road_shapefiles and kdtree available here

In [57]:
all_assignment = pd.read_csv('all_assignment.csv')

pfas_mlf = all_assignment[all_assignment['facility_label'] == 'MLF']
pfas_wts = all_assignment[all_assignment['facility_label'] == 'WTS']

C:\Users\hgazmeh\AppData\Local\Temp\2\ipykernel_18052\1159401190.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pfas_gdf_sub['facility_assigned'] = pfas_gdf_sub.apply(lambda x: assign_tree_facility(x['x'], x['y'], all_facilities_sub), axis=1)


waste transfer stations assignment to the nearest station 

In [83]:
def assign_tree_facility(x, y, landfills):
    landfill_tree = cKDTree(landfills[['x', 'y']])
    _, nearest_indices = landfill_tree.query([x, y], k=1)
    # # select the top 5 nearest landfills
    sub_landfills = landfills.iloc[nearest_indices]['facility_id']
    return sub_landfills

main_landfills_gdf['x'] = main_landfills_gdf.geometry.x
main_landfills_gdf['y'] = main_landfills_gdf.geometry.y

main_landfills_gdf_sub1 = main_landfills_gdf[['facility_id', 'x', 'y', 'facility_label']]
waste_transfer_stations['mlf_assigned'] = waste_transfer_stations.apply(lambda x: assign_tree_facility(x['x'], x['y'], main_landfills_gdf_sub1), axis=1)

pfas_wts = pfas_wts.merge(waste_transfer_stations[['facility_id', 'mlf_assigned']], on='facility_id', how='left') 

c:\Users\hgazmeh\Anaconda3\envs\myenv\lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


all pfas from cts are assigned

In [98]:
pfas_all_assigned = pd.concat([pfas_mlf, pfas_wts], axis=0)
pfas_all_assigned.reset_index(inplace=True, drop=True)
pfas_all_assigned['main_landfill_ID'] = pfas_all_assigned['mlf_assigned'].fillna(pfas_all_assigned['facility_id'])
# pfas_all_assigned.to_csv('pfas_all_assigned.csv', index=False)

get the facility files

In [112]:
pfas_gdf_sub_s1 = pfas_all_assigned[['main_landfill_ID', 'PFAS_CT_2020']].groupby('main_landfill_ID').sum().reset_index()
pfas_gdf_sub_s2 = pfas_wts[['facility_id', 'PFAS_CT_2020']].groupby('facility_id').sum().reset_index()

pfas_gdf_sub_s1 = main_landfills_gdf.merge(pfas_gdf_sub_s1, on='main_landfill_ID', how='left')
pfas_gdf_sub_s2 = waste_transfer_stations.merge(pfas_gdf_sub_s2, on='facility_label', how='left')

# pfas_gdf_sub_s1.to_csv('MLF_pfas_assigned.csv', index=False)
# pfas_gdf_sub_s2.to_csv('WTS_pfas_assigned.csv', index=False)